In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
import tensorflow as tf
from keras.metrics import Precision, Recall
from keras.layers import Reshape, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, Concatenate
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

2024-01-23 23:35:47.996394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from PIL import Image
import numpy as np
import os

def load_png_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            label = int(filename[0])  # Assuming the label is the first character in the filename
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img = img.resize(target_size)  # Resize the image to a consistent size
            images.append(np.array(img))
            labels.append(label)
    return np.array(images), np.array(labels)

# Replace 'path/to/your/images' with the actual path to your folder containing .png files
image_folder_path = 'img/urdu_numerals/'
images, labels = load_png_images_from_folder(image_folder_path)

# Print the shape of the created dataset
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (95, 224, 224)
Labels shape: (95,)


In [ ]:
np.savez("datasets/urdu_numericals.npz", image=images, label = labels)

In [3]:
model =tf.keras.models.load_model(
        "models/numerical_model_convolutional.h5"
    ) 

model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)               0     

In [4]:
labels_x = to_categorical(labels)

In [5]:
for m in model.layers:
    print(m.name)
new_model = Sequential()


conv2d_3
max_pooling2d_2
conv2d_4
max_pooling2d_3
conv2d_5
flatten_1
dense_2
dense_3


In [6]:
new_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [30]:
new_model = Sequential()
new_model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(224, 224,1)))
new_model.add(MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_newmodel'))   # Changed (3, 3) to (2, 2)

for idx, layer in enumerate(model.layers):
    if idx not in [0,5,6,7]:
        # Append the index to the layer name to make it unique
        new_layer_name = f"{layer.name}_new_{idx}"
        layer._name = new_layer_name
        layer.trainable = False
        new_model.add(layer)

new_model.add(Conv2D(32, (1, 1), activation='relu',name="Conv2D_2_new"))
new_model.add(MaxPooling2D(1, 1,name="maxpo0ling_new"))
new_model.add(Flatten(name="flatten_new"))
new_model.add(Dense(64, activation='softmax',name="dense_final_new"))
new_model.add(Dense(10, activation='softmax', name="dense_final_new_new"))

new_model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 223, 223, 32)      160       
                                                                 
 max_pooling2d_newmodel (Ma  (None, 111, 111, 32)      0         
 xPooling2D)                                                     
                                                                 
 max_pooling2d_2_new_1 (Max  multiple                  0         
 Pooling2D)                                                      
                                                                 
 conv2d_4_new_2_new_2_new_2  multiple                  18496     
 _new_2_new_2_new_2_new_2 (                                      
 Conv2D)                                                         
                                                                 
 max_pooling2d_3_new_3_new_  multiple                

In [31]:
for layer in new_model.layers:
    print(layer.trainable)

True
True
False
False
False
False
True
True
True
True
True


In [32]:
for layer in new_model.layers:
    print(layer)
var1 = new_model.layers[0].input

# new_model = tf.keras.Model(inputs=var1, outputs=final_layer)
#new_model.build()
new_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
new_model.layers[-1]

In [ ]:
new_model.summary()

In [34]:
new_model.fit(images,labels_x, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 2s 663ms/step - loss: 2.0653 - accuracy: 0.5263 - val_loss: 2.1054 - val_accuracy: 0.3158
Epoch 2/10
3/3 [==============================] - 1s 476ms/step - loss: 2.0587 - accuracy: 0.5263 - val_loss: 2.1251 - val_accuracy: 0.2632
Epoch 3/10
3/3 [==============================] - 1s 462ms/step - loss: 2.0522 - accuracy: 0.5263 - val_loss: 2.1378 - val_accuracy: 0.2632
Epoch 4/10
3/3 [==============================] - 1s 475ms/step - loss: 2.0476 - accuracy: 0.5132 - val_loss: 2.1347 - val_accuracy: 0.3684
Epoch 5/10
3/3 [==============================] - 1s 471ms/step - loss: 2.0430 - accuracy: 0.5789 - val_loss: 2.1193 - val_accuracy: 0.3684
Epoch 6/10
3/3 [==============================] - 1s 468ms/step - loss: 2.0387 - accuracy: 0.5789 - val_loss: 2.0951 - val_accuracy: 0.3684
Epoch 7/10
3/3 [==============================] - 1s 472ms/step - loss: 2.0342 - accuracy: 0.5789 - val_loss: 2.0799 - val_accuracy: 0.4211
Epoch 8/10
3/3 [====

In [ ]:
images.shape

In [35]:
new_model.save("models/numerical_model_convolutional_2_fine_tuned_urdu_224.h5")

/Users/rusab1/Work/learntorch/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
